In [1]:
!pip install datasets

  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
   ---------------------------------------- 0.0/507.1 kB ? eta -:--:--
   ---- ----------------------------------- 61.4/507.1 kB 3.2 MB/s eta 0:00:01
   ---- ----------------------------------- 61.4/507.1 kB 3.2 MB/s eta 0:00:01
   --------------- ------------------------ 194.6/507.1 kB 1.7 MB/s eta 0:00:01
   ---------------- ----------------------- 204.8/507.1 kB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 317.4/507.1 kB 1.6 MB/s eta 0:00:01
   -------------------------- ------------- 337.9/507.1 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------  501.8/507.1 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 507.1/507.1 kB 1.8 MB/s eta 0:00:00
Using cached dill-0.3.7-py3-none-any.whl (115 kB)
   ---------------------------------------- 0.0/364.8 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor
from torch.nn.utils.rnn import pad_sequence

In [3]:
# Defining the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads),
            num_layers
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)  # 
        x = self.fc(x)
        return x


In [4]:
# Loading the Iris dataset
iris_data = load_iris()
data = iris_data.data
targets = iris_data.target

In [5]:
# Dividing the training set and test set
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=42)

In [6]:
# Defining custom data sets
class IrisDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.long)
        return x, y

# Set random seeds
torch.manual_seed(42)

# Creating a data loader
train_dataset = IrisDataset(train_data, train_targets)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = IrisDataset(test_data, test_targets)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Define hyperparameters
input_dim = 4
hidden_dim = 128
output_dim = 3
num_layers = 2
num_heads = 4
batch_size = 16
num_epochs = 50
learning_rate = 0.001

# Create models and optimizers
model = TransformerModel(input_dim, hidden_dim, output_dim, num_layers, num_heads)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

c:\Users\USER\Wayne\Rolvaag\.venv\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
# Training Model
for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        
        # Adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)
        
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * total_correct / total_samples
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/50, Loss: 1.7092, Accuracy: 41.67%
Epoch 2/50, Loss: 0.7321, Accuracy: 61.67%
Epoch 3/50, Loss: 0.4303, Accuracy: 78.33%
Epoch 4/50, Loss: 0.4543, Accuracy: 73.33%
Epoch 5/50, Loss: 0.3111, Accuracy: 85.00%
Epoch 6/50, Loss: 0.1587, Accuracy: 93.33%
Epoch 7/50, Loss: 0.3031, Accuracy: 89.17%
Epoch 8/50, Loss: 0.1332, Accuracy: 95.00%
Epoch 9/50, Loss: 0.1686, Accuracy: 91.67%
Epoch 10/50, Loss: 0.2636, Accuracy: 88.33%
Epoch 11/50, Loss: 0.2387, Accuracy: 89.17%
Epoch 12/50, Loss: 0.2243, Accuracy: 90.83%
Epoch 13/50, Loss: 0.1862, Accuracy: 92.50%
Epoch 14/50, Loss: 0.1372, Accuracy: 94.17%
Epoch 15/50, Loss: 0.1851, Accuracy: 92.50%
Epoch 16/50, Loss: 0.2656, Accuracy: 92.50%
Epoch 17/50, Loss: 0.1303, Accuracy: 93.33%
Epoch 18/50, Loss: 0.1161, Accuracy: 94.17%
Epoch 19/50, Loss: 0.1609, Accuracy: 94.17%
Epoch 20/50, Loss: 0.2298, Accuracy: 86.67%
Epoch 21/50, Loss: 0.2058, Accuracy: 92.50%
Epoch 22/50, Loss: 0.1196, Accuracy: 95.00%
Epoch 23/50, Loss: 0.2049, Accuracy: 91.6

In [8]:
# Evaluating the model on a test set
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        # Adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 96.67%


In [9]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install transformers

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     -------- ---------------------------- 30.7/129.4 kB 262.6 kB/s eta 0:00:01
     -------- ---------------------------- 30.7/129.4 kB 262.6 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/129.4 kB 363.1 kB/s eta 0:00:01
     ---------------------------- ------- 102.4/129.4 kB 454.0 kB/s eta 0:00:01
     ------------------------------------ 129.4/129.4 kB 543.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.4 MB 2.6 MB/s eta 0:00:04
    --------------------------------------- 0.1/8.4 MB 1.6 MB/s eta 0:00:06
    --

In [11]:
# 检查训练集中的标签值范围
train_labels = [sample['label'] for sample in dataset['train']]
min_label = min(train_labels)
max_label = max(train_labels)
print("Min Label:", min_label)
print("Max Label:", max_label)

# 检查验证集中的标签值范围
val_labels = [sample['label'] for sample in dataset['test']]
min_label = min(val_labels)
max_label = max(val_labels)
print("Min Label:", min_label)
print("Max Label:", max_label)


NameError: name 'dataset' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from transformers import ViTModel

# 加载数据集
from datasets import load_dataset
dataset = load_dataset("alkzar90/CC6204-Hackaton-Cub-Dataset")

from torchvision.transforms import ToTensor
from PIL import Image

# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label = sample['label']

        # Resize the image while maintaining the aspect ratio
        image = self.resize_image(image, size=224)

        image = self.transform(image)

        return image, label

    @staticmethod
    def resize_image(image, size):
        # Resize the image while maintaining the aspect ratio
        width, height = image.size
        aspect_ratio = width / height
        if width > height:
            new_height = size
            new_width = int(new_height * aspect_ratio)
        else:
            new_width = size
            new_height = int(new_width / aspect_ratio)

        resized_image = image.resize((new_width, new_height), Image.BILINEAR)

        # Pad the image if necessary to achieve the desired size
        padded_image = Image.new("RGB", (size, size))
        padded_image.paste(resized_image, ((size - new_width) // 2, (size - new_height) // 2))

        return padded_image


# 划分训练集和验证集
transform = ToTensor()
train_dataset = CustomDataset(dataset['train'], transform)
val_dataset = CustomDataset(dataset['test'], transform)

# 创建数据加载器
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# 定义Transformer模型
class ImageTransformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ImageTransformer, self).__init__()
        self.transformer = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = self.transformer(x)
        x = x.last_hidden_state[:, 0, :]  # 提取CLS向量
        x = self.fc(x)
        return x

# 创建模型实例
input_dim = 768  # ViT模型的输出维度
num_classes = 200  # 分类的类别数
model = ImageTransformer(input_dim, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    for batch in train_loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = train_correct / train_total

    # Evaluate on the validation set
    model.eval()
    with torch.no_grad():
        val_correct = 0
        val_total = 0
        for batch in val_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch [1/10], Training Accuracy: 0.0032, Validation Accuracy: 0.0050
Epoch [2/10], Training Accuracy: 0.0038, Validation Accuracy: 0.0052
Epoch [3/10], Training Accuracy: 0.0027, Validation Accuracy: 0.0052
Epoch [4/10], Training Accuracy: 0.0047, Validation Accuracy: 0.0052
Epoch [5/10], Training Accuracy: 0.0043, Validation Accuracy: 0.0052
Epoch [6/10], Training Accuracy: 0.0042, Validation Accuracy: 0.0052
Epoch [7/10], Training Accuracy: 0.0057, Validation Accuracy: 0.0052
Epoch [8/10], Training Accuracy: 0.0042, Validation Accuracy: 0.0048
Epoch [9/10], Training Accuracy: 0.0037, Validation Accuracy: 0.0050
Epoch [10/10], Training Accuracy: 0.0047, Validation Accuracy: 0.0052
